In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# # dx = 250m; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
# res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
times=data['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
index_adjust=0
ocean_fraction=0.25

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
#LIMITING Y AXIS
limit_y=True
# limit_y=False

In [ ]:
#NEEDED TO PLOT THE CORRECT DATA #*#*
data_type="Tracked_Properties"

In [ ]:
def limit_axes_to_y(ax, y_min=0, y_max=7, buffer_frac=0.1):
    ax.set_ylim(y_min, y_max)

    x_limited = []
    for line in ax.get_lines():
        xdata, ydata = np.array(line.get_xdata()), np.array(line.get_ydata())
        # Mask for y within limits
        y_mask = (ydata >= y_min) & (ydata <= y_max)
        # Apply mask
        x_visible = xdata[y_mask]
        # Remove NaN or Inf from x_visible
        x_visible = x_visible[np.isfinite(x_visible)]
        x_limited.extend(x_visible)
    if len(x_limited) > 0:
        x_limited = np.array(x_limited)
        x_min, x_max = np.min(x_limited), np.max(x_limited)

        if not (np.isfinite(x_min) and np.isfinite(x_max)):
            print("Warning: Non-finite x-limits detected, skipping set_xlim")
            return
        x_range = x_max - x_min

        if x_range == 0:
            buffer = 0.1  # fixed small buffer
        else:
            buffer = buffer_frac * x_range
        ax.set_xlim(x_min - buffer, x_max + buffer)
    else:
        print("Warning: No visible x data within y limits to set xlim")


In [ ]:
# def PlotProfile(type1): #ORIGINAL CODE WITHOUT VMF_c and VMF_g combined
    
#     profiles = []
#     SE_profiles = []
#     for var, label in var_units:
#         profiles.append((
#             globals()[f"{type1}_ALL_profile_array_{var.upper()}"].copy(),
#             globals()[f"{type1}_SHALLOW_profile_array_{var.upper()}"].copy(),
#             globals()[f"{type1}_DEEP_profile_array_{var.upper()}"].copy(),
#             label
#         ))
        
#         SE_profiles.append((
#             globals()[f"{type1}_ALL_profile_array_{var.upper()}_SE"].copy(),
#             globals()[f"{type1}_SHALLOW_profile_array_{var.upper()}_SE"].copy(),
#             globals()[f"{type1}_DEEP_profile_array_{var.upper()}_SE"].copy(),
#             None  # placeholder for xlabel not used in SE
#         ))
    
#     # Set up the 2x3 gridspec
#     fig = plt.figure(figsize=(12, 8))
#     gs = gridspec.GridSpec(3, 3, figure=fig)
#     lw=0.8
    
#     # Loop through the profiles and plot them on subplots
#     for i, (var,(ALL_profile_array, SHALLOW_profile_array, DEEP_profile_array, xlabel), (ALL_SE, SHALLOW_SE, DEEP_SE, _)) in enumerate(zip(var_units,profiles, SE_profiles)):
#         ax = fig.add_subplot(gs[i])
        
     
#         # Plot Profiles
#         profile_list=[ALL_profile_array,SHALLOW_profile_array,DEEP_profile_array]
#         SE_profile_list=[ALL_SE,SHALLOW_SE,DEEP_SE]
#         colors=['black','green','blue']
#         labels=['ALL','SHALLOW','DEEP']
#         # for (profile,color,label) in zip(profile_list,colors,labels):   
#         for (profile,SE_profile,color,label) in zip(profile_list,SE_profile_list,colors,labels):   
#             profile=averaged_profiles(profile) # Compute the averaged profile
                    
#             ################################################
#             if xlabel in [r'$q_v$ (g/kg)',r'$q_c+q_i$ (g/kg)','HMC (g/kg/s)']:
#                 profile[:,0]*=1000
#                 SE_profile[:,0]*=1000
#                 switch=1; factor=1; min_value=0
#             else:
#                 switch=1; factor=1; min_value=None
#             #STANDARD ERROR SHADING
#             PlotSE(ax,profile,SE_profile,color=color,factor=factor,min_value=min_value)
#             ################################################
            
#             ax.plot(profile[:, 0], profile[:, 1],color=color,label=label,lw=lw)
            
    
            
#         ax.set_xlabel(xlabel)
#         ax.set_ylabel('z (km)')
#         ax.grid(True)
    
#         #LEGEND
#         if i==1:
#             ax.legend(fontsize='small')
    
#         if i in [6,7,8]:
#             apply_scientific_notation([ax])
            
#         #LIMITING YAXIS TO BELOW 7 KM
#         if limit_y==True: 
#             # #LIMITING YAXIS TO BELOW 7 KM
#             limit_axes_to_y(ax,y_min=0, y_max=7)
    
#     #MEAN CLOUD BASE
#     hline_lw=1.2
#     axs = fig.get_axes()
#     for axis in axs:
#         axis.axhline(all_cloudbase,color='purple',linestyle='dashed',lw=hline_lw)
#         axis.axhline(MeanLFC/1000,color='green',linestyle='dashed',lw=hline_lw)

#     # #FIXING X AXISES
#     # ax_lst=[axs[7],axs[8]]
#     # MatchAxisLimits(ax_lst,dim='x')
     
#     #LEGEND
#     # legend_ax = fig.add_subplot(gs[1, 2])  # Use the (2, 3) grid slot for the legend
#     # legend_ax.axis("off")  # Hide axes for the legend box
#     # legend_ax.legend(*ax.get_legend_handles_labels(), loc='center', frameon=False)

#     #ACCESSORIES
#     plt.suptitle(f'(ALL, SHALLOW < 4 km, DEEP > 6 km) Tracked Lagrangian {type1} (BL-LFC to Exit Cloudy Updraft) Parcels')
#     plt.tight_layout()

In [ ]:
def PlotProfile(type1):
    
    profiles = []
    SE_profiles = []
    for var, label in var_units:
        profiles.append((
            globals()[f"{type1}_ALL_profile_array_{var.upper()}"].copy(),
            globals()[f"{type1}_SHALLOW_profile_array_{var.upper()}"].copy(),
            globals()[f"{type1}_DEEP_profile_array_{var.upper()}"].copy(),
            label
        ))
        
        SE_profiles.append((
            globals()[f"{type1}_ALL_profile_array_{var.upper()}_SE"].copy(),
            globals()[f"{type1}_SHALLOW_profile_array_{var.upper()}_SE"].copy(),
            globals()[f"{type1}_DEEP_profile_array_{var.upper()}_SE"].copy(),
            None
        ))
    
    fig = plt.figure(figsize=(12, 8))
    gs = gridspec.GridSpec(3, 3, figure=fig)
    lw=0.8
    
    i = 0  # subplot counter

    # Save vmf_g axis so vmf_c can use it later
    vmf_g_ax = None

    for idx, (var, (ALL_profile_array, SHALLOW_profile_array, DEEP_profile_array, xlabel), (ALL_SE, SHALLOW_SE, DEEP_SE, _)) in enumerate(zip(var_units, profiles, SE_profiles)):
        
        var_name = var[0]
        
        # Handle vmf_g and vmf_c together
        if var_name == 'vmf_c':
            # Plot vmf_c on vmf_g's axis
            if vmf_g_ax is None:
                raise ValueError("Cannot plot vmf_c before vmf_g.")
            ax = vmf_g_ax
        else:
            ax = fig.add_subplot(gs[i])
            if var_name == 'vmf_g':
                vmf_g_ax = ax  # Save this axis for vmf_c
            i += 1  # Increment subplot index only for new subplots

        # Plot profiles
        profile_list = [ALL_profile_array, SHALLOW_profile_array, DEEP_profile_array]
        SE_profile_list = [ALL_SE, SHALLOW_SE, DEEP_SE]
        colors = ['black', 'green', 'blue']
        labels = ['ALL', 'SHALLOW', 'DEEP']

        for (profile, SE_profile, color, label1) in zip(profile_list, SE_profile_list, colors, labels):
            profile = averaged_profiles(profile)
            
            if xlabel in [r'$q_v$ (g/kg)', r'$q_c+q_i$ (g/kg)', 'HMC (g/kg/s)']:
                profile[:,0] *= 1000
                SE_profile[:,0] *= 1000
                min_value=0
            elif var[0] in ['MSE']:
                cpd=1005.7
                #should divide by cpd + r_T*cl, but r_T is not available here
                #assume r_T is on average < 1e-3 g/kg 
                #so cl term would only contribute < 4 to the divisor
                profile[:,0] /= cpd
                SE_profile[:,0] /= cpd
            else:
                min_value=None

            PlotSE(ax, profile, SE_profile, color=color, factor=1, min_value=min_value)
            ax.plot(profile[:, 0], profile[:, 1], color=color, linestyle='solid', label=f"{var_name} {label1}", lw=lw)

        ax.set_xlabel(xlabel)
        ax.set_ylabel('z (km)')
        ax.grid(True)

        if i == 2:
            ax.legend(fontsize='small')

        # if i in [6, 7, 8]:
        #     apply_scientific_notation([ax])

        if limit_y:
            limit_axes_to_y(ax, y_min=0, y_max=7)

    # Mean Cloud Base Lines
    hline_lw=1.2
    axs = fig.get_axes()
    for axis in axs:
        axis.axhline(all_cloudbase, color='purple', linestyle='dashed', lw=hline_lw)
        axis.axhline(MeanLFC/1000, color='green', linestyle='dashed', lw=hline_lw)

    plt.suptitle(f'(ALL, SHALLOW < 4 km, DEEP > 6 km) Tracked Lagrangian {type1} (BL-LFC to Exit Cloudy Updraft) Parcels')
    plt.tight_layout()


In [ ]:
# def PlotDifference(type1,type2): #ORIGINAL CODE WITHOUT VMF_c and VMF_g combined
    
#     # Construct profiles dynamically using globals()
#     profiles = [
#         (
#             globals()[f"{type1}_{type2}_ALL_diff_{var.upper()}"].copy(),
#             globals()[f"{type1}_{type2}_SHALLOW_diff_{var.upper()}"].copy(),
#             globals()[f"{type1}_{type2}_DEEP_diff_{var.upper()}"].copy(),
#             label
#         )
#         for var, label in var_units
#     ]
    
#     # Set up the 2x3 gridspec
#     fig = plt.figure(figsize=(12, 8))
#     gs = gridspec.GridSpec(3, 3, figure=fig)
#     lw=0.8
    
#     # Loop through the profiles and plot them on subplots
#     for i, (ALL_profile_array, SHALLOW_profile_array, DEEP_profile_array, xlabel) in enumerate(profiles):
#         ax = fig.add_subplot(gs[i])
    
        
#         # Compute the averaged profile
#         ALL_profile = ALL_profile_array.copy()
#         SHALLOW_profile = SHALLOW_profile_array.copy()
#         DEEP_profile = DEEP_profile_array.copy()
    
#         if xlabel in [r'$q_v$ (g/kg)',r'$q_c+q_i$ (g/kg)','HMC (g/kg/s)']:
#             ALL_profile[:,0]*=1000
#             SHALLOW_profile[:,0]*=1000
#             DEEP_profile[:,0]*=1000
        
#         # Plot the profile
#         ax.plot(ALL_profile[:, 0], ALL_profile[:, 1],color='black',label=f'ALL {type1}-{type2}',lw=lw)
#         ax.plot(SHALLOW_profile[:, 0], SHALLOW_profile[:, 1],color='green',label=f'SHALLOW {type1}-{type2}',lw=lw)
#         ax.plot(DEEP_profile[:, 0], DEEP_profile[:, 1],color='blue',label=f'DEEP {type1}-{type2}',lw=lw)
#         ax.set_xlabel(xlabel)
#         ax.set_ylabel('z (km)')
#         ax.grid(True)
    
#         ax.axvline(0,color='k',linestyle='dashed')
    
#         #LEGEND
#         if i==1:
#             ax.legend(fontsize='small')
    
#         # if i in [6,7,8]:
#         apply_scientific_notation([ax])
        
#         #LIMITING YAXIS TO BELOW 7 KM
#         if limit_y==True: 
#             # #LIMITING YAXIS TO BELOW 7 KM
#             limit_axes_to_y(ax,y_min=0, y_max=7)
    
#     #MEAN CLOUD BASE
#     hline_lw=1.2
#     axs = fig.get_axes()  # axs is already a list of axes
#     for ax in axs:
#         print(ax)
#         ax.axhline(all_cloudbase, color='purple', linestyle='dashed',lw=hline_lw)
#         ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed',lw=hline_lw)

#     # #FIXING X AXISES
#     # ax_lst=[axs[7],axs[8]]
#     # MatchAxisLimits(ax_lst,dim='x')
    
#     # #LEGEND
#     # legend_ax = fig.add_subplot(gs[1, 2])  # Use the (2, 3) grid slot for the legend
#     # legend_ax.axis("off")  # Hide axes for the legend box
#     # legend_ax.legend(*ax.get_legend_handles_labels(), loc='center', frameon=False)
    
#     #ACCESSORIES
#     plt.suptitle(f'(ALL, SHALLOW < 4 km, DEEP > 6 km) Tracked Lagrangian {type1}-{type2} Difference (BL-LFC to Exit Cloudy Updraft) Parcels')
#     plt.tight_layout()
    


In [ ]:
def PlotDifference(type1, type2):
    
    # Construct profiles dynamically using globals()
    profiles = [
        (
            globals()[f"{type1}_{type2}_ALL_diff_{var.upper()}"].copy(),
            globals()[f"{type1}_{type2}_SHALLOW_diff_{var.upper()}"].copy(),
            globals()[f"{type1}_{type2}_DEEP_diff_{var.upper()}"].copy(),
            label
        )
        for var, label in var_units
    ]
    
    fig = plt.figure(figsize=(12, 8))
    gs = gridspec.GridSpec(3, 3, figure=fig)
    lw = 0.8

    i = 0  # subplot counter
    vmf_g_ax = None  # Placeholder for vmf_g axis to reuse with vmf_c

    for idx, ((var, label), (ALL_profile_array, SHALLOW_profile_array, DEEP_profile_array, xlabel)) in enumerate(zip(var_units, profiles)):
        
        var_name = var  # Assuming var is a string

        if var_name == 'vmf_c':
            if vmf_g_ax is None:
                raise ValueError("Cannot plot vmf_c before vmf_g.")
            ax = vmf_g_ax  # Reuse vmf_g axis
            linestyle = '-'  # Dashed line for vmf_c
        else:
            ax = fig.add_subplot(gs[i])
            if var_name == 'vmf_g':
                vmf_g_ax = ax  # Save vmf_g axis for vmf_c
            linestyle = '-'  # Solid line for other variables
            i += 1

        # Get copies of the profiles
        ALL_profile = ALL_profile_array.copy()
        SHALLOW_profile = SHALLOW_profile_array.copy()
        DEEP_profile = DEEP_profile_array.copy()

        if xlabel in [r'$q_v$ (g/kg)', r'$q_c+q_i$ (g/kg)', 'HMC (g/kg/s)']:
            ALL_profile[:, 0] *= 1000
            SHALLOW_profile[:, 0] *= 1000
            DEEP_profile[:, 0] *= 1000

        # Plot differences
        ax.plot(ALL_profile[:, 0], ALL_profile[:, 1], color='black', linestyle=linestyle, label=f'ALL {type1}-{type2}', lw=lw)
        ax.plot(SHALLOW_profile[:, 0], SHALLOW_profile[:, 1], color='green', linestyle=linestyle, label=f'SHALLOW {type1}-{type2}', lw=lw)
        ax.plot(DEEP_profile[:, 0], DEEP_profile[:, 1], color='blue', linestyle=linestyle, label=f'DEEP {type1}-{type2}', lw=lw)

        ax.set_xlabel(xlabel)
        ax.set_ylabel('z (km)')
        ax.grid(True)
        ax.axvline(0, color='k', linestyle='dashed')

        # Legend
        if i == 2:
            ax.legend(fontsize='small')

        if i in []:
            apply_scientific_notation([ax])

        # Y-axis limits
        if limit_y:
            limit_axes_to_y(ax, y_min=0, y_max=7)

    # Mean Cloud Base lines
    hline_lw = 1.2
    axs = fig.get_axes()
    for ax in axs:
        ax.axhline(all_cloudbase, color='purple', linestyle='dashed', lw=hline_lw)
        ax.axhline(MeanLFC / 1000, color='green', linestyle='dashed', lw=hline_lw)

    # Title and layout
    plt.suptitle(f'(ALL, SHALLOW < 4 km, DEEP > 6 km) Tracked Lagrangian {type1}-{type2} Difference (BL-LFC to Exit Cloudy Updraft) Parcels')
    plt.tight_layout()


In [ ]:
##################################
#PLOTTING

In [ ]:
def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
all_cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {min_all_cloudbase}\n")

In [ ]:
def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")

In [ ]:
def averaged_profiles(profile):
    out_var=profile[ (profile[:, 1] > 1)]; #gets rid of rows that have no data\n"
    out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
    return out_var

In [ ]:
# List of variables with their corresponding labels and x-axis titles
var_units = [
    ("qv", r"$q_v$ (g/kg)"),
    ("qcqi", r"$q_c+q_i$ (g/kg)"),
    ("RH_VAPOR", r"$RH_v$ (fraction)"),
    # ("RH_ICE", r"$RH_i$ (fraction)"),

    ("w", 'w (m/s)'),
    ("vmf_g", "VMF (g/kg/s)"), ("vmf_c", "VMF (g/kg/s)"),
    ("hmc", "HMC (g/kg/s)"),
    
    # ("th", r"$\theta$ (K)"),
    ("th_v", r"$\theta_v$ (K)"),
    ("th_e", r"$\theta_e$ (K)"),
    ("MSE", r"$MSE$ (K)"),
    
    # ("buoyancy_cm1", r"$B_{cm1}$ $(m/s^2)$"),
    # ("buoyancy_full", r"$B_{full}$ $(m/s^2)$"),
    # ("buoyancy_full_each_t", r"$B_{full\_each\_time}$ $(m/s^2)$"),
    
]

In [ ]:
def PlotSE(ax, profile, SE_profile, color, factor, switch=1, alpha=0.1, min_value=None):
    lower = profile[:, 0] - factor * SE_profile[:, 0] * switch
    upper = profile[:, 0] + factor * SE_profile[:, 0] * switch
    
    if min_value is not None:
        lower = np.maximum(lower, min_value)
    ax.fill_betweenx(profile[:, -1], lower, upper, color=color, alpha=alpha)
    #Example #PlotSE(ax,profile,SE_profile,color=color,factor=factor, min_value=min_value)

In [ ]:
########################################################################
#CL vs nonCL

In [ ]:
type1='CL';type2='nonCL'

for data_type2 in [data_type,"Tracked_Entrainment_VMF"]:
        
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    filePath=dir3+f"{data_type2}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    key_list=[]
    with h5py.File(filePath, 'r') as h5f:
        keys=h5f.keys()
        for key in keys:
            globals()[key] = h5f[key][:]
            if '_squares' not in key:
                key_list.append(key)
        # print(keys)
    
    #CALCULATING STANDARD DEVIATION
    try:
        for key in key_list:
            # globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
            globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
            # print(key)
    except:
        #*#*#
        #IF SE NOT AVAILABLE, MAKE DUMMY VARIABLES
        for key in key_list:
            dummy=globals()[key].copy()
            dummy[:,0]=np.zeros(len(dummy[:,0]))
            dummy=dummy[dummy[:,1]>1]
            globals()[key+f"_SE"]=dummy

In [ ]:
type1='CL'
PlotProfile(type1)

In [ ]:
type1='nonCL'
PlotProfile(type1)

In [ ]:
#DIFFERENCES

def average_difference(array1, array2):
    out_var_one=averaged_profiles(array1)
    out_var_two=averaged_profiles(array2)

    #masking out non matches
    second_col_one = out_var_one[:, 1]
    second_col_two = out_var_two[:, 1]
    mask_one = np.isin(second_col_one, second_col_two)
    mask_two = np.isin(second_col_two, second_col_one)
    
    out_var_one = out_var_one.copy()[mask_one]
    out_var_two = out_var_two.copy()[mask_two]
    
    diff=(out_var_one[:,0]-out_var_two[:,0])
    zs=out_var_one[:,1]

    out_profile=np.zeros((len(diff),2))

    out_profile[:,0]=diff;out_profile[:,1]=zs;
    return out_profile

var_names = [var[0] for var in var_units]
categories = ["ALL", "SHALLOW", "DEEP"]

for var in var_names:
    for cat in categories:
        # Construct the variable names dynamically
        CL_profile = globals()[f"CL_{cat}_profile_array_{var.upper()}"]
        nonCL_profile = globals()[f"nonCL_{cat}_profile_array_{var.upper()}"]
        
        # Calculate the difference and assign it to the global variable
        globals()[f"CL_nonCL_{cat}_diff_{var.upper()}"] = average_difference(CL_profile, nonCL_profile)
print('done')

In [ ]:
type1='CL';type2='nonCL'
PlotDifference(type1,type2)

In [ ]:
#SBZ vs nonSBZ
################################################################################

In [ ]:
##################################
#PLOTTING

In [ ]:
type1='SBZ';type2='nonSBZ'

for data_type2 in [data_type,"Tracked_Entrainment_VMF"]:
        
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    filePath=dir3+f"{data_type2}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    key_list=[]
    with h5py.File(filePath, 'r') as h5f:
        for key in h5f.keys():
            globals()[key] = h5f[key][:]
            if '_squares' not in key:
                key_list.append(key)
    
    #CALCULATING STANDARD DEVIATION
    try:
        for key in key_list:
            # globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
            globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
            # print(key)
    except:
        #*#*#
        #IF SE NOT AVAILABLE, MAKE DUMMY VARIABLES
        for key in key_list:
            dummy=globals()[key].copy()
            dummy[:,0]=np.zeros(len(dummy[:,0]))
            dummy=dummy[dummy[:,1]>1]
            globals()[key+f"_SE"]=dummy

In [ ]:
type1='SBZ'
PlotProfile(type1)

In [ ]:
type1='nonSBZ'
PlotProfile(type1)

In [ ]:
#SBZ vs nonSBZ
def average_difference(array1, array2):
    out_var_one=averaged_profiles(array1)
    out_var_two=averaged_profiles(array2)

    #masking out non matches
    second_col_one = out_var_one[:, 1]
    second_col_two = out_var_two[:, 1]
    mask_one = np.isin(second_col_one, second_col_two)
    mask_two = np.isin(second_col_two, second_col_one)
    
    out_var_one = out_var_one.copy()[mask_one]
    out_var_two = out_var_two.copy()[mask_two]
    
    diff=(out_var_one[:,0]-out_var_two[:,0])
    zs=out_var_one[:,1]

    out_profile=np.zeros((len(diff),2))

    out_profile[:,0]=diff;out_profile[:,1]=zs;
    return out_profile
    

var_names = [var[0] for var in var_units]
categories = ["ALL", "SHALLOW", "DEEP"]

for var in var_names:
    for cat in categories:
        # Construct the variable names dynamically
        CL_profile = globals()[f"SBZ_{cat}_profile_array_{var.upper()}"]
        nonCL_profile = globals()[f"nonSBZ_{cat}_profile_array_{var.upper()}"]
        
        # Calculate the difference and assign it to the global variable
        globals()[f"SBZ_nonSBZ_{cat}_diff_{var.upper()}"] = average_difference(CL_profile, nonCL_profile)
print('done')

In [ ]:
type1='SBZ';type2='nonSBZ'
PlotDifference(type1,type2)

In [ ]:
#ColdPool
################################################################

In [ ]:
#PLOTTING
############################################################

In [ ]:
# type1='ColdPool'

# for data_type2 in [data_type,"Tracked_Entrainment_VMF"]:
#     dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
#     filePath=dir3+f"{data_type2}_"+f"{type1}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
#     key_list=[]
#     with h5py.File(filePath, 'r') as h5f:
#         for key in h5f.keys():
#             globals()[key] = h5f[key][:]
#             if '_squares' not in key:
#                 key_list.append(key)
    
#     #CALCULATING STANDARD DEVIATION
#     try:
#         for key in key_list:
#             globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
#             # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
#             # print(key)
#     except:
#         #*#*#
#         #IF SE NOT AVAILABLE, MAKE DUMMY VARIABLES
#         for key in key_list:
#             dummy=globals()[key].copy()
#             dummy[:,0]=np.zeros(len(dummy[:,0]))
#             dummy=dummy[dummy[:,1]>1]
#             globals()[key+f"_SE"]=dummy

In [ ]:
# type1='ColdPool'
# PlotProfile(type1)

In [ ]:
# #SBZ vs ColdPool
# def average_difference(array1, array2):
#     out_var_one=averaged_profiles(array1)
#     out_var_two=averaged_profiles(array2)

#     #masking out non matches
#     second_col_one = out_var_one[:, 1]
#     second_col_two = out_var_two[:, 1]
#     mask_one = np.isin(second_col_one, second_col_two)
#     mask_two = np.isin(second_col_two, second_col_one)
    
#     out_var_one = out_var_one.copy()[mask_one]
#     out_var_two = out_var_two.copy()[mask_two]
    
#     diff=(out_var_one[:,0]-out_var_two[:,0])
#     zs=out_var_one[:,1]

#     out_profile=np.zeros((len(diff),2))

#     out_profile[:,0]=diff;out_profile[:,1]=zs;
#     return out_profile
    
# # Calculate average differences for each variable (w, qv, qc, th, buoyancy) for ColdPool

# var_names = [var[0] for var in var_units]
# categories = ["ALL", "SHALLOW", "DEEP"]

# for var in var_names:
#     for cat in categories:
#         # Construct the variable names dynamically
#         CL_profile = globals()[f"SBZ_{cat}_profile_array_{var.upper()}"]
#         nonCL_profile = globals()[f"ColdPool_{cat}_profile_array_{var.upper()}"]
        
#         # Calculate the difference and assign it to the global variable
#         globals()[f"SBZ_ColdPool_{cat}_diff_{var.upper()}"] = average_difference(CL_profile, nonCL_profile)
# print('done')

In [ ]:
# type1='SBZ';type2='ColdPool'
# PlotDifference(type1,type2)

In [ ]:
################################################
#TESTING

In [ ]:
# import matplotlib.pyplot as plt

# # Plot with double x-axis
# fig, ax1 = plt.subplots()

# # First x-axis for W
# test_var = CL_ALL_profile_array_W
# out1 = averaged_profiles(test_var)
# line1, = ax1.plot(out1[:, 0], out1[:, 1], 'k-', label='W (m/s)')
# ax1.set_xlabel('W (m/s)', color='k')
# ax1.set_ylabel('z (km)')
# ax1.tick_params(axis='x', labelcolor='k')

# # Twin x-axis for QI and QCQI
# #1
# ax2 = ax1.twiny()
# test_var = CL_ALL_profile_array_QI
# out2 = averaged_profiles(test_var)
# line2, = ax2.plot(out2[:, 0]*1000, out2[:, 1], 'b-', label='QI (g/kg)')
# #2
# test_var = CL_ALL_profile_array_QCQI
# out3 = averaged_profiles(test_var)
# line3, = ax2.plot(out3[:, 0]*1000, out3[:, 1], 'b--', label='QC+QI (g/kg)')
# ax2.set_xlabel('Q (g/kg)', color='b')
# ax2.tick_params(axis='x', labelcolor='b')
# #3
# test_var = CL_ALL_profile_array_QR
# out3 = averaged_profiles(test_var)
# line3, = ax2.plot(out3[:, 0]*1000, out3[:, 1], 'g-', label='QR (g/kg)')
# ax2.tick_params(axis='x', labelcolor='b')

# # Combine all lines for a single legend
# lines = [line1, line2, line3]
# labels = [line.get_label() for line in lines]

# plt.title('ALL CL Tracked Profiles: W and QI (and QCQI) vs z')
# plt.legend(lines, labels, loc='lower right')  # Adjust loc as needed
# plt.show()


In [ ]:
# import matplotlib.pyplot as plt

# # Plot with double x-axis
# fig, ax1 = plt.subplots()

# # First x-axis for W
# test_var = CL_ALL_profile_array_BUOYANCY
# out1 = averaged_profiles(test_var)
# line1, = ax1.plot(out1[:, 0], out1[:, 1], 'k-', label='W (m/s)')
# ax1.set_xlabel('W (m/s)', color='k')
# ax1.set_ylabel('z (km)')
# ax1.tick_params(axis='x', labelcolor='k')

# # Twin x-axis for QI and QCQI
# ax2 = ax1.twiny()
# test_var = CL_ALL_profile_array_TH_E
# out2 = averaged_profiles(test_var)
# line2, = ax2.plot(out2[:, 0], out2[:, 1], 'b-', label='QI (g/kg)')

# ax2.set_xlabel('TH (K)', color='b')
# ax2.tick_params(axis='x', labelcolor='b')

# # Combine all lines for a single legend
# lines = [line1, line2, line3]
# labels = [line.get_label() for line in lines]

# plt.title('ALL CL Tracked Profiles: BUOYANCY and TH vs z')
# plt.legend(lines, labels, loc='lower center')  # Adjust loc as needed
# plt.show()
